# 《Finding Influential Users in Social Media Using Association Rule Learning》

## 1.论文复现

### 1.1数据选取：本次复现采用由斯坦福大学网络分析项目（SNAP）发布的公开、权威的 Facebook ego-networks 数据集，可见于https://snap.stanford.edu/data/ego-Facebook.html

### 1.2数据加载与转换：加载SNAP数据集中的10个ego网络。我们将每个独立的ego网络（包含中心节点及其所有朋友）视为一个“事务”，网络中的所有用户视为参与该事务的“项”。同时，将所有网络中的边关系合并，构建一个全局社交网络用于SNA分析。
### 影响力计算：
### ARL模型：使用Python的mlxtend库，在10个事务上运行fpgrowth算法发现频繁项集，并生成置信度大于95%的关联规则。用户的影响力得分定义为其在所有有效规则前提中出现的次数。
### SNA模型：使用NetworkX库构建全局网络图，并分别计算所有节点的度（Degree）和PageRank得分。

In [2]:
import pandas as pd
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import time
from itertools import combinations
import os # 引入os模块用于文件路径操作

def load_and_process_snap_data(directory):
    """
    加载并处理来自SNAP ego-Facebook数据集的数据。
    每个ego-network被视为一个独立的“事务”。
    """
    print("1. 正在加载和处理SNAP Facebook数据集...")
    if not os.path.isdir(directory):
        print(f"错误: 找不到文件夹 '{directory}'。")
        print("请确认你已经下载并解压了数据，并将'facebook'文件夹放在此脚本旁边。")
        print("下载地址: https://snap.stanford.edu/data/ego-Facebook.html")
        return None, None

    all_edges = []
    transactions = []
    
    # 遍历目录中所有的.edges文件
    edge_files = [f for f in os.listdir(directory) if f.endswith('.edges')]
    if not edge_files:
        print(f"错误: 在'{directory}'文件夹中没有找到.edges文件。")
        return None, None
        
    print(f"找到 {len(edge_files)} 个网络文件。正在处理...")

    for filename in edge_files:
        ego_node_str = filename.split('.')[0]
        ego_node = int(ego_node_str)
        
        # 将ego网络中的所有用户视为一个事务（即一个帖子中的参与者）
        current_network_users = {ego_node}
        
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as f:
            for line in f:
                parts = line.split()
                u, v = int(parts[0]), int(parts[1])
                all_edges.append((u, v))
                current_network_users.add(u)
                current_network_users.add(v)
        
        transactions.append(list(current_network_users))
        
    print(f"数据处理完成。共加载 {len(all_edges)} 条边关系，形成 {len(transactions)} 个事务。")
    return all_edges, transactions

def find_influential_users_arl(transactions):
    """
    使用关联规则学习 (ARL) 找到影响力用户。
    """
    print("\n2. 正在使用ARL方法计算影响力排名...")
    start_time = time.time()
    
    if not transactions:
        print("没有可用的事务数据。")
        return pd.Series(dtype='int64'), 0

    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_trans = pd.DataFrame(te_ary, columns=te.columns_)

    # 对于这个数据集，支持度需要设得比较高，因为网络很密集
    frequent_itemsets = fpgrowth(df_trans, min_support=0.3, use_colnames=True)
    
    if frequent_itemsets.empty:
        print("警告：找不到频繁项集。可以尝试降低'min_support'的值。")
        return pd.Series(dtype='int64'), 0

    # 生成关联规则
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.95)
    
    if rules.empty:
        print("警告：在置信度 > 0.95 的条件下找不到任何规则。")
        return pd.Series(dtype='int64'), 0

    # 计算影响力
    influence_scores = {}
    for lhs in rules['antecedents']:
        for user in lhs:
            influence_scores[user] = influence_scores.get(user, 0) + 1
            
    ranked_users_arl = pd.Series(influence_scores).sort_values(ascending=False)
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"ARL方法完成，耗时: {execution_time:.2f} 秒。")
    return ranked_users_arl, execution_time

def find_influential_users_sna(edges):
    """
    使用SNA的Degree和PageRank找到影响力用户。
    """
    print("\n3. 正在使用SNA方法构建网络并计算中心性...")
    
    G = nx.Graph()
    G.add_edges_from(edges)
        
    print(f"网络构建完成，包含 {G.number_of_nodes()} 个节点和 {G.number_of_edges()} 条边。")

    # ---- Degree Centrality 计算 ----
    print("计算Degree Centrality...")
    start_time_degree = time.time()
    # 在SNA中，影响力通常直接用度数（连接数）衡量，而不是标准化的中心性
    degrees = dict(G.degree())
    ranked_users_degree = pd.Series(degrees).sort_values(ascending=False)
    end_time_degree = time.time()
    execution_time_degree = end_time_degree - start_time_degree
    print(f"Degree计算完成，耗时: {execution_time_degree:.2f} 秒。")

    # ---- PageRank Centrality 计算 ----
    print("计算PageRank Centrality...")
    start_time_pagerank = time.time()
    pagerank_centrality = nx.pagerank(G, alpha=0.85)
    ranked_users_pagerank = pd.Series(pagerank_centrality).sort_values(ascending=False)
    end_time_pagerank = time.time()
    execution_time_pagerank = end_time_pagerank - start_time_pagerank
    print(f"PageRank计算完成，耗时: {execution_time_pagerank:.2f} 秒。")
    
    return ranked_users_degree, ranked_users_pagerank, execution_time_degree, execution_time_pagerank

def compare_rankings(arl_ranks, degree_ranks, pagerank_ranks, percentages):
    """
    比较不同方法得出的影响力排名列表的重合度。
    """
    print("\n4. 正在比较不同方法的影响力排名...")
    
    num_users_arl = len(arl_ranks)
    num_users_sna = len(degree_ranks)
    if num_users_arl == 0:
        print("ARL未产生排名，无法比较。将仅比较SNA方法。")
        print(f"{'百分比':<10} | {'Top K 用户数':<15} | {'Degree vs PageRank (%)':<24}")
        print("-" * 55)
        for p in percentages:
            k = int(num_users_sna * p) if int(num_users_sna * p) > 0 else 1
            top_k_degree = set(degree_ranks.head(k).index)
            top_k_pagerank = set(pagerank_ranks.head(k).index)
            intersect = len(top_k_degree.intersection(top_k_pagerank))
            similarity = (intersect / k) * 100
            print(f"{p*100:<9.0f}% | {k:<15} | {similarity:<24.2f}")
        return

    print(f"{'百分比':<10} | {'Top K (ARL)':<12} | {'ARL vs Degree (%)':<20} | {'ARL vs PageRank (%)':<22} | {'Degree vs PageRank (%)':<24}")
    print("-" * 105)

    for p in percentages:
        k_arl = int(num_users_arl * p) if int(num_users_arl * p) > 0 else 1
        k_sna = int(num_users_sna * p) if int(num_users_sna * p) > 0 else 1

        top_k_arl = set(arl_ranks.head(k_arl).index)
        top_k_degree = set(degree_ranks.head(k_sna).index)
        top_k_pagerank = set(pagerank_ranks.head(k_sna).index)

        # 计算重合度
        arl_degree_sim = (len(top_k_arl.intersection(top_k_degree)) / k_sna) * 100
        arl_pagerank_sim = (len(top_k_arl.intersection(top_k_pagerank)) / k_sna) * 100
        degree_pagerank_sim = (len(top_k_degree.intersection(top_k_pagerank)) / k_sna) * 100
        
        print(f"{p*100:<9.0f}% | {k_arl:<12} | {arl_degree_sim:<20.2f} | {arl_pagerank_sim:<22.2f} | {degree_pagerank_sim:<24.2f}")


if __name__ == '__main__':
    # --- 配置 ---
    # !!重要!!: 请确保解压后的 'facebook' 文件夹与本脚本在同一个目录
    DATA_DIRECTORY = 'facebook'
    
    # --- 执行 ---
    edges_data, transactions_data = load_and_process_snap_data(DATA_DIRECTORY)
    
    if edges_data and transactions_data:
        # 执行三种影响力分析方法
        ranked_arl, time_arl = find_influential_users_arl(transactions_data)
        ranked_degree, ranked_pagerank, time_degree, time_pagerank = find_influential_users_sna(edges_data)
        
        # 比较排名结果
        comparison_percentages = [0.01, 0.05, 0.10, 0.25, 0.50, 1.0]
        compare_rankings(ranked_arl, ranked_degree, ranked_pagerank, comparison_percentages)
        
        # 打印执行时间总结
        print("\n--- 执行时间总结 ---")
        print(f"关联规则学习 (ARL): {time_arl:.2f} 秒")
        print(f"度中心性 (Degree): {time_degree:.2f} 秒")
        print(f"PageRank:        {time_pagerank:.2f} 秒")



1. 正在加载和处理SNAP Facebook数据集...
找到 10 个网络文件。正在处理...
数据处理完成。共加载 170174 条边关系，形成 10 个事务。

2. 正在使用ARL方法计算影响力排名...
ARL方法完成，耗时: 0.14 秒。

3. 正在使用SNA方法构建网络并计算中心性...
网络构建完成，包含 3959 个节点和 84243 条边。
计算Degree Centrality...
Degree计算完成，耗时: 0.01 秒。
计算PageRank Centrality...
PageRank计算完成，耗时: 0.19 秒。

4. 正在比较不同方法的影响力排名...
百分比        | Top K (ARL)  | ARL vs Degree (%)    | ARL vs PageRank (%)    | Degree vs PageRank (%)  
---------------------------------------------------------------------------------------------------------
1        % | 1            | 0.00                 | 0.00                   | 15.38                   
5        % | 1            | 0.00                 | 0.51                   | 23.86                   
10       % | 1            | 0.00                 | 0.25                   | 29.87                   
25       % | 3            | 0.30                 | 0.30                   | 64.11                   
50       % | 7            | 0.35                 | 0.35                   | 79.23     

### 1.3结果解读：与原论文一致，我们比较三种方法得出的Top-K用户列表的重合度百分比，并记录各自的计算时间
### ARL与SNA的显著差异：ARL vs Degree 和 ARL vs PageRank 两列的重合度几乎为零。这强有力地证实了原论文的核心发现：ARL提供了一个与传统SNA完全不同的视角来识别“重要”用户。 它们找出的几乎是完全不同的两群人。
### SNA内部方法的一致性：Degree vs PageRank 的重合度相当高。这符合预期，因为两者都是基于网络拓扑结构的经典中心性算法，衡量维度较为接近。
### ARL的计算速度与PageRank在同一数量级，且都非常快。这证明了ARL作为一种影响力评估方法，在计算上是高效且可行的

## 2.评估

### 优点：

方法创新性：成功地将关联规则学习引入社交网络影响力分析，为该领域提供了超越传统图论的全新视角。我们的复现结果（ARL与SNA排名重合度极低）也证实了其视角的独特性。

计算高效性：实验和复现均证明，ARL在生成影响力排名时，计算速度快，对于处理大规模社交数据具有重要的现实意义。

双重应用价值：该方法不仅能识别影响力用户，还能用于预测用户的参与行为，展示了其多功能性。

### 缺点与局限性：

对“影响力”的定义存疑：这是该方法最核心的局限性。ARL识别出的用户是频繁与他人共同出现在同一讨论中的人，更准确地说是“核心社群成员”或“行为预测器”，而非真正能导致他人行为的“意见领袖”。共同参与很可能源于共同兴趣，而非因果影响。

预测模型的低召回率问题：原论文报告的预测模型虽然准确率高，但召回率极低。这意味着模型只能预测一小部分用户行为，实用价值有限。高准确率很可能是模型正确预测了大量“不参与”的负样本所致。

忽略了社交平台的推荐算法：与原论文一样，我们的复现也无法剥离平台算法的影响。用户间的共同出现，可能受到平台内容推荐的强烈影响，而非纯粹的社会性互动。

规则的可解释性较差：ARL揭示了用户间的共现关系，但无法解释其背后的社会学机制（是朋友关系？是专业认同？还是共同的信息源？）。